In [ ]:
# Als het weer nodig is, check je huidige working directory
import os
print(os.getcwd())

# Pas je working directory aan indien gewenst
path = 'C:\\Users\\rburrus\\Desktop\\Jupyter Notebook' # zet hier wat voor jou goed is
os.chdir(path)
print(os.getcwd())

In [ ]:
# Importeer relevante pakketjes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None) # maakt onbeperkt aantal kolommen zichtbaar

## 1. 
**Importeer de dataset Honda.csv**

In [ ]:
df_honda=pd.read_csv("Honda")

## 2. 
**a. Inspecteer de waarde Massa.ledig.voertuig middels een histogram. Zijn er waardes die als
outlier gezien kunnen worden? Verwijder deze. Doe hetzelfde voor de variabele
Massa.rijklaar** <br>
Video data inspecteren op outliers: https://youtu.be/Gxlco3CCR9I

In [ ]:
df_honda."Massa.ledig.voertuig".plot.hist()


In [ ]:
# Plot een boxplot
        plt.boxplot(df_honda["Massa.ledig.voertuig"])
Plt.title("Massa.ledig.voertuig")
        plt.show() 

kolom = df_honda["Massa.ledig.voertuig"]

q1 = kolom.quantile(0.25)
q3 = kolom.quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

uitschieters = df_honda[(kolom < lower) | (kolom > upper)]
print("Aantal uitschieters:", len(uitschieters))
print(uitschieters[["Massa.ledig.voertuig"]])
print("Q1:", q1)
print("Q3:", q3)
print("IQR:", iqr)
print("Lower bound:", lower)
print("Upper bound:", upper)

In [ ]:
df_clean = df_honda.loc[(df_honda["Massa.ledig.voertuig"] > 116) & (df_honda["Massa.ledig.voertuig"] < 308)]



In [ ]:
# Kan ook de tweede hoogste waarde uithalen


In [ ]:
# Check de histogram weer
df_clean["Massa.ledig.voertuig"].plot.hist()

In [ ]:

kolom = df_clean["Massa.rijklaar"]

q1 = kolom.quantile(0.25)
q3 = kolom.quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

uitschieters = df_clean[(kolom < lower) | (kolom > upper)]
print("Aantal uitschieters:", len(uitschieters))
print(uitschieters[["Massa.rijklaar"]])
print("Q1:", q1)
print("Q3:", q3)
print("IQR:", iqr)
print("Lower bound:", lower)
print("Upper bound:", upper)


df_further = df_clean.loc[(df_clean["Massa.rijklaar"] > 129.5) & (df_clean["Massa.rijklaar"] < 309.5)] 


In [ ]:
df_further["Massa.rijklaar"].plot.hist()



**b. Inspecteer waardes middels een 2-d plot voor de variabelen massa.ledig.voertuig en
massa.rijklaar . Welke additionele waarde(s) zou je nog kunnen verwijderen?**

In [ ]:
df_further.plot(x="Massa.ledig.voertuig", y="Massa.rijklaar", kind="scatter")


In [ ]:
mask = df_further["Massa.rijklaar"] <= 2 * df_further["Massa.ledig.voertuig"]
df_cleaned = df_further.loc[mask]

## 3. 
**a. Zet de variabele Datum_tenaamstelling in Date format om.**
Video: https://www.youtube.com/watch?v=4WLtGZ-mbjc&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=13 

In [ ]:
type(df_further["Datum_tenaamstelling"])


In [ ]:
# Maak gebruik van assign() en pd.to_datetime om de variabele om te zetten
df_further = df_further.assign( tenaamstelling = pd.to_datetime(df_further["Datum_tenaamstelling"]))
print(df_further["Datum_tenaamstelling"])  


## 4. 
**Controleer of er NA-waardes in de variabele Bruto.BPM zitten.** <br>
Video behandelen missing data: https://youtu.be/Gxlco3CCR9I?t=502 <br>

**a. Maak een nieuwe variabele flag, die 1 is indien Bruto.BPM een NA-waarde bevat, en 0 wanneer er input is.**

In [ ]:
df_further["Bruto.BPM"].isna()



In [ ]:
# Maak 'flag' met gebruik van assign(), np.where() en isna() 


# Bekijk de head van Bruto.BPM en je nieuwe kolom te controleren of het gelukt is
df_further = df_further.assign(
    Bruto_BPM_missing_flag = np.where(df_further["Bruto.BPM"].isna(), 1, 0)
)

print(df_further[["Bruto.BPM", "Bruto_BPM_missing_flag"]].head(10))


**b. Vul de NA-waardes in met**

i. Gemiddelde van de waarde

ii. Mediaan van de waarde 

iii. Interpolatie van bovenstaande onderstaande waarde

**Let op: om deze drie methoden in rij te kunnen doen, moet je eerst een kopie van je dataframe maken (anders wordt alle NaN's ingevuld met je gemiddelde en dan ziet de volgende stap geen NaN's meer)**

In [ ]:
# i. Gemiddelde: maak direct gebruik van fillna(), dus heb je 'flag' niet nodig
df_further["Bruto.BPM"] = df_further["Bruto.BPM"].fillna(df_further["Bruto.BPM"].mean())

In [ ]:
# ii. Mediaan: maak direct gebruik van fillna(), dus heb je 'flag' niet nodig
mediaan_BPM = df_further["Bruto.BPM"].median()
print(mediaan_BPM)


In [ ]:
# iii. Interpolatie: maak direct gebruik van fillna(), dus heb je 'flag' niet nodig
df_further["Bruto.BPM"] = df_further["Bruto.BPM"].interpolate(method="linear")


In [ ]:
# Vergelijk de verschillen tussen de drie methoden


## 5. 
**a. Maak een nieuwe dataset met per unieke waarde voor de variabele Handelsbenaming het
aantal keren dat deze voorkomt. Welke benaming komt het vaakste voor? **

In [ ]:
from collections import Counter
Unieke_benamingen= Counter(df_further['Handelsbenaming'])
print(Unieke_benamingen)

**b. Selecteer uit het oorspronkelijke dataframe alleen de rijen waarin de string “CBR 600” voorkomt in de Handelsbenaming. Kies ervoor om zowel hoofd als kleine letters toe te laten**

Tip: https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html

In [ ]:
CBR_600 = df_further[df_further["Handelsbenaming"].str.contains("CBR 600", case=False, na=False)].copy()
print ( CBR_600)

**c. Bonus: Vervang de waardes in de nieuwe dataset die heel erg lijken (maximaal verschil van 20%
waardes) op de meest voorkomende waarde.** <br>
Tip: Gebruik fuzz.ratio uit fuzzywuzzy <br>
Tip2: om fuzz.ratio toe te kunnen passen op een kolom moet je gebruik maken van de apply functie

Video fuzzywuzzy: https://www.youtube.com/watch?v=aE2AwoBxXeU&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=19 <br>
Video apply: https://www.youtube.com/watch?v=eYVD3q15tsA&list=PLwkTCAI_gJjW10xaAJpdqeQx7EQgFolfG&index=18

In [ ]:
# Check voor de meest voorkomende waardes voor 'Handelsbenaming' binnen je nieuwe dataset


In [ ]:
# Installeer en importeer de volgende:
# %pip install fuzzywuzzy
#of
#!conda install -c conda-forge fuzzywuzzy

# from fuzzywuzzy import fuzz

# Probeer eens fuzz.ratio() met een van de handelsbenamingen en de meest voorkomende
# Dit geeft een ratio van overeenkomst


# Nu doe hetzelfde met fuzz.partial_ratio()
# (inhoeverre er een gedeeltelijke overeenkomst is (substring))


In [ ]:
# Maak een nieuwe kolom 'Ratio' (via assign/apply/en fuzz.ratio)...
#...voor een berekening voor de fuzz ratio tegen de meest voorkomende waarde


# Soorteer Handelsbenaming en Ratio op Ratio, en maak gebruik van drop_duplicates


In [ ]:
# Vervang alle namen met een ratio >= 60 met de meest voorkomende waarde

# Print alle unieke waardes na de vervanging
